In [1]:
import pandas as pd
import numpy as np
import h3
import basedosdados as bd

In [2]:
pontos_interesse_df = pd.read_csv("csvs\pontos_de_alagamento_top50 - Pontos_de_Alagamento.csv", sep=',', decimal=',') # lendo o arquivo csv
res = 8 # resolução do h3

# convertendo lat e lon para float
pontos_interesse_df.loc[1:,('lat')] = pontos_interesse_df.loc[1:,('lat')].str.replace(',', '.').astype(float) 
pontos_interesse_df.loc[1:,('lon')] = pontos_interesse_df.loc[1:,('lon')].str.replace(',', '.').astype(float)
latlon = pontos_interesse_df[['lat', 'lon']][1:].values.astype(float)
h3_pontos_interesse = np.array([h3.geo_to_h3(lat, lon, res) for lat, lon in latlon]) # encontrar o hexagono para cada ponto de interesse
mask = h3_pontos_interesse == h3_pontos_interesse[0] # criar uma mascara para os pontos de interesse que estão no mesmo hexagono do primeiro ponto de interesse
pontos_interesse_df.loc[1:,("endereco")][mask] # mostrar os pontos de interesse que estão no mesmo hexagono do primeiro ponto de interesse

1                Av. Presidente Vargas, Cidade Nova
24    Ponto 7 - Haddock lobo, ambos os lados da via
Name: endereco, dtype: object

In [4]:
query = "SELECT o.id_pop, o.data_inicio, o.data_fim, o.latitude, o.longitude, o.gravidade  FROM `datario.adm_cor_comando.ocorrencias` as o WHERE o.id_pop in (31,5,32,6) "
proj_id = "projeto-fgv1"
alagamento_df = bd.read_sql(query, billing_project_id=proj_id)
alagamento_df.head()

Downloading: 100%|██████████|


,id_pop,data_inicio,data_fim,latitude,longitude,gravidade
0,5,2017-02-02 19:30:32+00:00,2017-02-03 01:00:10+00:00,-23.010140,-43.442512,Baixo
1,5,2017-02-08 01:30:34+00:00,2017-02-08 03:05:52+00:00,-22.885083,-43.389702,Baixo
2,5,2020-10-30 00:21:07+00:00,2020-10-30 20:04:52+00:00,-22.850693,-43.471502,Baixo
3,5,2020-10-07 23:52:01+00:00,2020-10-08 00:24:16+00:00,-22.979718,-43.202439,Baixo
4,5,2020-10-30 18:09:50+00:00,2020-10-30 21:53:25+00:00,-22.970484,-43.217380,Baixo


In [5]:
h3_pontos_interesse[0]

'88a8a06a5bfffff'

In [6]:
alagamento_df["h3"] = alagamento_df.apply(lambda row: h3.geo_to_h3(row["latitude"],row["longitude"], res),axis=1)
alagamento_df.head()

,id_pop,data_inicio,data_fim,latitude,longitude,gravidade,h3
0,5,2017-02-02 19:30:32+00:00,2017-02-03 01:00:10+00:00,-23.010140,-43.442512,Baixo,88a8a07469fffff
1,5,2017-02-08 01:30:34+00:00,2017-02-08 03:05:52+00:00,-22.885083,-43.389702,Baixo,88a8a06729fffff
2,5,2020-10-30 00:21:07+00:00,2020-10-30 20:04:52+00:00,-22.850693,-43.471502,Baixo,88a8a06451fffff
3,5,2020-10-07 23:52:01+00:00,2020-10-08 00:24:16+00:00,-22.979718,-43.202439,Baixo,88a8a078cbfffff
4,5,2020-10-30 18:09:50+00:00,2020-10-30 21:53:25+00:00,-22.970484,-43.217380,Baixo,88a8a078d1fffff


In [11]:
alagamento_interesse = alagamento_df[alagamento_df["h3"]==h3_pontos_interesse[0]]
alagamento_interesse.shape

(27, 7)

In [12]:
alagamento_interesse.to_csv("csvs/alalamentos_ponto_interesse.csv",index=False)
alagamento_df.to_csv("csvs/alagamento.csv",index=False)